In [27]:
import pandas as pd
import numpy as np

# Lire les fichiers de données
train_df = pd.read_csv('data/train.csv')
client_df = pd.read_csv('data/client.csv')
electricity_prices_df = pd.read_csv('data/electricity_prices.csv')
gas_prices_df = pd.read_csv('data/gas_prices.csv')

In [28]:
train_df_sample = train_df.sample(80000)


In [29]:
train_df_sample

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
827354,13,0,1,94.204,0,2022-05-25 13:00:00,266,827354,50
345348,11,0,3,0.000,0,2021-12-25 03:00:00,115,345348,45
1770844,11,1,1,0.110,0,2023-03-14 22:00:00,559,1770844,46
1125988,15,0,3,6.055,0,2022-08-27 06:00:00,360,1125988,58
1305612,10,0,1,0.015,0,2022-10-21 21:00:00,415,1305612,38
...,...,...,...,...,...,...,...,...,...
1577946,3,0,1,0.483,0,2023-01-13 13:00:00,499,1577946,11
776267,9,0,1,10.361,1,2022-05-09 16:00:00,250,776267,34
338731,2,1,3,368.618,1,2021-12-22 23:00:00,112,338731,10
846244,11,1,3,91.898,0,2022-05-31 10:00:00,272,846244,48


In [30]:
electricity_prices_df["date"] = pd.to_datetime(electricity_prices_df["forecast_date"])
electricity_prices_df["date"] = electricity_prices_df["date"].apply(lambda x : x.strftime("%d-%m-%y"))
electricity_prices_df["date"]

0        01-09-21
1        01-09-21
2        01-09-21
3        01-09-21
4        01-09-21
           ...   
15281    30-05-23
15282    30-05-23
15283    30-05-23
15284    30-05-23
15285    30-05-23
Name: date, Length: 15286, dtype: object

In [31]:
client_df["date_str"] = client_df["date"].apply(lambda x : str(x))
electricity_prices_df["date_str"] = electricity_prices_df["date"].apply(lambda x : str(x))

In [32]:
electricity_prices_df

,forecast_date,euros_per_mwh,origin_date,data_block_id,date,date_str
0,2021-09-01 00:00:00,92.51,2021-08-31 00:00:00,1,01-09-21,01-09-21
1,2021-09-01 01:00:00,88.90,2021-08-31 01:00:00,1,01-09-21,01-09-21
2,2021-09-01 02:00:00,87.35,2021-08-31 02:00:00,1,01-09-21,01-09-21
3,2021-09-01 03:00:00,86.88,2021-08-31 03:00:00,1,01-09-21,01-09-21
4,2021-09-01 04:00:00,88.43,2021-08-31 04:00:00,1,01-09-21,01-09-21
...,...,...,...,...,...,...
15281,2023-05-30 19:00:00,82.10,2023-05-29 19:00:00,637,30-05-23,30-05-23
15282,2023-05-30 20:00:00,150.85,2023-05-29 20:00:00,637,30-05-23,30-05-23
15283,2023-05-30 21:00:00,82.10,2023-05-29 21:00:00,637,30-05-23,30-05-23
15284,2023-05-30 22:00:00,82.09,2023-05-29 22:00:00,637,30-05-23,30-05-23


In [33]:
train_df["price_elec"] = -1

for unique_date in train_df["datetime"].unique()[:10]:
  # Récuperer le prix de l'electricite a ce jour la
  df_elec_filtered = electricity_prices_df[electricity_prices_df["origin_date"] == unique_date]
  if len(df_elec_filtered) != 0:
    if len(df_elec_filtered) == 1:
      elec_price = df_elec_filtered["euros_per_mwh"]
    else : 
      elec_price = df_elec_filtered.iloc[0]["euros_per_mwh"]

    # Pour toutes les lignes de df_train a ce jour la, insérer le prix
    df_train_filtered = train_df[train_df["datetime"] == unique_date]
    df_train_filtered.loc[:,"price_elec"] = elec_price

In [34]:
train_df.describe()

,county,is_business,product_type,target,is_consumption,data_block_id,row_id,prediction_unit_id,price_elec
count,2.018352e+06,2.018352e+06,2.018352e+06,2.017824e+06,2018352.0,2.018352e+06,2.018352e+06,2.018352e+06,2018352.0
mean,7.297034e+00,5.368261e-01,1.898927e+00,2.748556e+02,0.5,3.218746e+02,1.009176e+06,3.304538e+01,-1.0
std,4.780990e+00,4.986421e-01,1.081766e+00,9.095024e+02,0.5,1.826343e+02,5.826482e+05,1.959059e+01,0.0
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,-1.0
25%,3.000000e+00,0.000000e+00,1.000000e+00,3.780000e-01,0.0,1.660000e+02,5.045878e+05,1.600000e+01,-1.0
50%,7.000000e+00,1.000000e+00,2.000000e+00,3.113300e+01,0.5,3.230000e+02,1.009176e+06,3.300000e+01,-1.0
75%,1.100000e+01,1.000000e+00,3.000000e+00,1.802062e+02,1.0,4.790000e+02,1.513763e+06,5.000000e+01,-1.0
max,1.500000e+01,1.000000e+00,3.000000e+00,1.548027e+04,1.0,6.370000e+02,2.018351e+06,6.800000e+01,-1.0


In [35]:
def find_time(group):
  electricity_prices_df[electricity_prices_df["origin_date"] == group.iloc[0]["datetime"]].iloc[0]["euros_per_mwh"]
  train_df.groupby("datetime")["electricity price"] = train_df.groupby("datetime").apply(find_time)

In [37]:
import time

def find_electricity_price(line):
    if int(line.name) % 2000 == 0:
        print(line.name, "/", train_df_sample.shape[0],
              (line.name / train_df_sample.shape[0]) * 100, "%", end="\r")
        time.sleep(2)
    filtered_df = electricity_prices_df[electricity_prices_df["origin_date"] == line["datetime"]]
    if not filtered_df.empty:
        return filtered_df.iloc[0]["euros_per_mwh"]
    else:
        return np.nan

train_df_sample["euros_per_mwh"] = train_df_sample.apply(find_electricity_price, axis=1)
train_df_sample


,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,euros_per_mwh
827354,13,0,1,94.204,0,2022-05-25 13:00:00,266,827354,50,149.99
345348,11,0,3,0.000,0,2021-12-25 03:00:00,115,345348,45,91.58
1770844,11,1,1,0.110,0,2023-03-14 22:00:00,559,1770844,46,81.67
1125988,15,0,3,6.055,0,2022-08-27 06:00:00,360,1125988,58,82.02
1305612,10,0,1,0.015,0,2022-10-21 21:00:00,415,1305612,38,143.00
...,...,...,...,...,...,...,...,...,...,...
1577946,3,0,1,0.483,0,2023-01-13 13:00:00,499,1577946,11,79.04
776267,9,0,1,10.361,1,2022-05-09 16:00:00,250,776267,34,111.21
338731,2,1,3,368.618,1,2021-12-22 23:00:00,112,338731,10,98.85
846244,11,1,3,91.898,0,2022-05-31 10:00:00,272,846244,48,100.49


In [38]:
df_elec_filtered

,forecast_date,euros_per_mwh,origin_date,data_block_id,date,date_str
33,2021-09-02 09:00:00,130.8,2021-09-01 09:00:00,2,02-09-21,02-09-21


In [39]:
electricity_prices_df

,forecast_date,euros_per_mwh,origin_date,data_block_id,date,date_str
0,2021-09-01 00:00:00,92.51,2021-08-31 00:00:00,1,01-09-21,01-09-21
1,2021-09-01 01:00:00,88.90,2021-08-31 01:00:00,1,01-09-21,01-09-21
2,2021-09-01 02:00:00,87.35,2021-08-31 02:00:00,1,01-09-21,01-09-21
3,2021-09-01 03:00:00,86.88,2021-08-31 03:00:00,1,01-09-21,01-09-21
4,2021-09-01 04:00:00,88.43,2021-08-31 04:00:00,1,01-09-21,01-09-21
...,...,...,...,...,...,...
15281,2023-05-30 19:00:00,82.10,2023-05-29 19:00:00,637,30-05-23,30-05-23
15282,2023-05-30 20:00:00,150.85,2023-05-29 20:00:00,637,30-05-23,30-05-23
15283,2023-05-30 21:00:00,82.10,2023-05-29 21:00:00,637,30-05-23,30-05-23
15284,2023-05-30 22:00:00,82.09,2023-05-29 22:00:00,637,30-05-23,30-05-23


In [40]:
train_df_sample["price_elec"] = -1

for unique_date in train_df_sample["datetime"].unique()[:10]:
  # Récuperer le prix de l'electricite a ce jour la
  df_elec_filtered = electricity_prices_df[electricity_prices_df["origin_date"] == unique_date]
  if len(df_elec_filtered) != 0:
    if len(df_elec_filtered) == 1:
      elec_price = df_elec_filtered["euros_per_mwh"]
    else : 
      elec_price = df_elec_filtered.iloc[0]["euros_per_mwh"]

    # Pour toutes les lignes de df_train a ce jour la, insérer le prix
    df_train_filtered = train_df_sample[train_df_sample["datetime"] == unique_date]
    df_train_filtered.loc[:,"price_elec"] = elec_price

In [41]:
train_df_sample.describe()

,county,is_business,product_type,target,is_consumption,data_block_id,row_id,prediction_unit_id,euros_per_mwh,price_elec
count,80000.000000,80000.000000,80000.000000,79987.000000,80000.000000,80000.000000,8.000000e+04,80000.000000,79744.000000,80000.0
mean,7.311300,0.534212,1.895300,273.366208,0.503425,322.635225,1.011607e+06,33.062863,157.564961,-1.0
std,4.783077,0.498831,1.080533,911.245725,0.499991,182.739574,5.830248e+05,19.568352,118.211458,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000e+00,0.000000,-10.060000,-1.0
25%,3.000000,0.000000,1.000000,0.394000,0.000000,166.000000,5.061165e+05,16.000000,85.250000,-1.0
50%,7.000000,1.000000,2.000000,31.644000,1.000000,324.000000,1.012996e+06,33.000000,129.305000,-1.0
75%,11.000000,1.000000,3.000000,181.474000,1.000000,480.000000,1.517861e+06,50.000000,200.000000,-1.0
max,15.000000,1.000000,3.000000,15207.541000,1.000000,637.000000,2.018339e+06,68.000000,4000.000000,-1.0


In [42]:
print(train_df_sample.head())

         county  is_business  product_type  target  is_consumption  \
827354       13            0             1  94.204               0   
345348       11            0             3   0.000               0   
1770844      11            1             1   0.110               0   
1125988      15            0             3   6.055               0   
1305612      10            0             1   0.015               0   

                    datetime  data_block_id   row_id  prediction_unit_id  \
827354   2022-05-25 13:00:00            266   827354                  50   
345348   2021-12-25 03:00:00            115   345348                  45   
1770844  2023-03-14 22:00:00            559  1770844                  46   
1125988  2022-08-27 06:00:00            360  1125988                  58   
1305612  2022-10-21 21:00:00            415  1305612                  38   

         euros_per_mwh  price_elec  
827354          149.99          -1  
345348           91.58          -1  
1770844    

In [43]:
train_df_sample["euros_per_mwh"]

827354     149.99
345348      91.58
1770844     81.67
1125988     82.02
1305612    143.00
            ...  
1577946     79.04
776267     111.21
338731      98.85
846244     100.49
1406429    300.06
Name: euros_per_mwh, Length: 80000, dtype: float64

In [44]:
train_df_sample.to_csv("./train_df_sample.csv")